In [32]:
import numpy as np

from mlscorecheck.check import (check_scores,
                                check_kfold_som_scores,
                                check_multiple_datasets_som_kfold_som_scores)

from mlscorecheck.core import (score_functions_with_solutions,
                                load_solutions,
                                Interval,
                                IntervalUnion)

from mlscorecheck.utils import (generate_problem,
                                generate_n_problems,
                                calculate_scores_som,
                                generate_folding_problem,
                                calculate_all_scores)

In [33]:
help(check_scores)

Help on function check_scores in module mlscorecheck.check._check_scores:

check_scores(scores, eps, dataset, *, return_details=False)
    Use for one dataset - based on the solution formulas



In [34]:
k = 4
eps = 10**(-k)

problem = generate_problem()

problem, eps

({'p': 392, 'n': 572, 'tp': 65, 'tn': 294, 'fp': 278, 'fn': 327}, 0.0001)

In [35]:
scores = calculate_all_scores(problem, rounding_decimals=k)
scores

{'acc': 0.3724,
 'sens': 0.1658,
 'spec': 0.514,
 'npv': 0.4734,
 'ppv': 0.1895,
 'bacc': 0.3399,
 'f1p': 0.1769,
 'fm': 0.1773}

In [36]:
flag, details = check_scores(scores, eps, problem, return_details=True)

2023-08-14 12:30:25,408:INFO:checking acc and sens against spec
2023-08-14 12:30:25,416:INFO:checking acc and sens against npv
2023-08-14 12:30:25,421:INFO:checking acc and sens against ppv
2023-08-14 12:30:25,424:INFO:checking acc and sens against bacc
2023-08-14 12:30:25,429:INFO:checking acc and sens against f1p
2023-08-14 12:30:25,431:INFO:checking acc and sens against fm
2023-08-14 12:30:25,434:INFO:checking acc and spec against sens
2023-08-14 12:30:25,436:INFO:checking acc and spec against npv
2023-08-14 12:30:25,442:INFO:checking acc and spec against ppv
2023-08-14 12:30:25,444:INFO:checking acc and spec against bacc
2023-08-14 12:30:25,446:INFO:checking acc and spec against f1p
2023-08-14 12:30:25,448:INFO:checking acc and spec against fm
2023-08-14 12:30:25,451:INFO:checking acc and npv against sens
2023-08-14 12:30:25,454:INFO:checking acc and npv against spec
2023-08-14 12:30:25,456:INFO:checking acc and npv against ppv
2023-08-14 12:30:25,462:INFO:checking acc and npv agai

In [37]:
details

{'succeeded': [{'details': [{'score_0': 'acc',
     'score_0_interval': (0.3723, 0.3725),
     'score_1': 'sens',
     'score_1_interval': (0.16570000000000001, 0.1659),
     'target_score': 'spec',
     'target_interval': (0.5139, 0.5141),
     'solution': {'tp': (64.9544, 65.0328),
      'tn': (293.8644, 294.1356),
      'tp_formula': 'p*sens',
      'tn_formula': 'acc*n + acc*p - p*sens'},
     'consistency': True,
     'explanation': 'the target score interval ((0.5139, 0.5141)) and the reconstructed intervals ((0.513748951048951, 0.5142230769230769)) do intersect',
     'target_interval_reconstructed': (0.513748951048951,
      0.5142230769230769)}],
   'consistency': True},
  {'details': [{'score_0': 'acc',
     'score_0_interval': (0.3723, 0.3725),
     'score_1': 'sens',
     'score_1_interval': (0.16570000000000001, 0.1659),
     'target_score': 'npv',
     'target_interval': (0.4733, 0.4735),
     'solution': {'tp': (64.9544, 65.0328),
      'tn': (293.8644, 294.1356),
      

In [38]:
for tmp in details['succeeded']:
    assert tmp['consistency']
    for item in tmp['details']:
        if item.get('target_interval_reconstructed') is not None and item.get('consistency'):
            target_interval = item['target_interval_reconstructed']
            iun = IntervalUnion(target_interval)
            print(iun, scores[item['target_score']])
            assert not iun.intersection(IntervalUnion(item['target_interval'])).is_empty()

(0.513748951048951, 0.5142230769230769) 0.514
(0.47307355728087064, 0.4737767858465967) 0.4734
(0.18927850000174845, 0.18970021480735597) 0.1895
(0.3397244755244755, 0.3400615384615384) 0.3399
(0.1767061805159199, 0.17700363681495354) 0.1769
(0.17709728244750034, 0.17740142512544918) 0.1773
(0.1653877551020408, 0.1661714285714286) 0.1658
(0.4731730371139211, 0.47367864904586193) 0.4734
(0.18891081400187182, 0.19003940871753047) 0.1895
(0.33964387755102043, 0.3401357142857143) 0.3399
(0.17636839754272507, 0.17730577919416074) 0.1769
(0.17675840982049135, 0.17770514914166569) 0.1773
(0.1555494248261108, 0.17762587793607867) 0.1658
(0.5099494185589686, 0.5170265140519853) 0.514
(0.17424608448298887, 0.2064705461547764) 0.1895
(0.3327494216925397, 0.34732619599403197) 0.3399
(0.16436778218303646, 0.1909651312973887) 0.1769
(0.164632555163099, 0.19150590598904205) 0.1773
(0.1655993091054838, 0.16603120266147695) 0.1658
(0.5132254062743439, 0.514726290236825) 0.514
(0.4723425797242166, 0.474

In [39]:
help(check_kfold_som_scores)

Help on function check_kfold_som_scores in module mlscorecheck.check._check_scores:

check_kfold_som_scores(scores, eps, *, dataset_setup, return_details=False)
    Use when scores are aggregated in the SoM manner



In [40]:
folds, problem = generate_folding_problem()

In [41]:
problem

{'p': 245, 'n': 623, 'n_folds': 5, 'n_repeats': 1}

In [42]:
scores = calculate_scores_som(folds)

In [43]:
scores

{'acc': 0.6417050691244239,
 'sens': 0.4163265306122449,
 'spec': 0.7303370786516854,
 'bacc': 0.5733318046319651}

In [44]:
check_kfold_som_scores(scores, 1e-4, dataset_setup=problem, return_details=True)

2023-08-14 12:30:25,938:INFO:checking acc and sens against spec
2023-08-14 12:30:25,941:INFO:checking acc and sens against bacc
2023-08-14 12:30:25,942:INFO:checking acc and spec against sens
2023-08-14 12:30:25,943:INFO:checking acc and spec against bacc
2023-08-14 12:30:25,945:INFO:checking acc and bacc against sens
2023-08-14 12:30:25,946:INFO:checking acc and bacc against spec
2023-08-14 12:30:25,948:INFO:checking sens and spec against acc
2023-08-14 12:30:25,950:INFO:checking sens and spec against bacc
2023-08-14 12:30:25,951:INFO:checking sens and bacc against acc
2023-08-14 12:30:25,953:INFO:checking sens and bacc against spec
2023-08-14 12:30:25,954:INFO:checking spec and bacc against acc
2023-08-14 12:30:25,955:INFO:checking spec and bacc against sens


(True,
 {'succeeded': [{'details': [{'score_0': 'acc',
      'score_0_interval': (0.6416050691244239, 0.6418050691244239),
      'score_1': 'sens',
      'score_1_interval': (0.4162265306122449, 0.4164265306122449),
      'target_score': 'spec',
      'target_interval': (0.7302370786516854, 0.7304370786516854),
      'solution': {'tp': (101.9755, 102.0245),
       'tn': (454.8887, 455.1112999999999),
       'tp_formula': 'p*sens',
       'tn_formula': 'acc*n + acc*p - p*sens'},
      'consistency': True,
      'explanation': 'the target score interval ((0.7302370786516854, 0.7304370786516854)) and the reconstructed intervals ((0.7301584269662921, 0.7305157303370785)) do intersect',
      'target_interval_reconstructed': (0.7301584269662921,
       0.7305157303370785)}],
    'consistency': True},
   {'details': [{'score_0': 'acc',
      'score_0_interval': (0.6416050691244239, 0.6418050691244239),
      'score_1': 'sens',
      'score_1_interval': (0.4162265306122449, 0.4164265306122449

In [45]:
for tmp in details['succeeded']:
    assert tmp['consistency']
    for item in tmp['details']:
        if item.get('target_interval_reconstructed') is not None and item.get('consistency'):
            target_interval = item['target_interval_reconstructed']
            iun = IntervalUnion(target_interval)
            assert not iun.intersection(IntervalUnion(item['target_interval'])).is_empty()